En el presente notebook se da paso a paso ejemplos de como usar la modelo YOLO de Ultralytics. La documentación se encuentra en github: https://github.com/ultralytics/yolov5

# Libreria

In [1]:
# librerias

import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
import cv2
import time
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Descargar pesos de modelo Yolo

El modelo se descargará desde github: https://github.com/ultralytics/yolov5/releases

In [ ]:
# Leer modelo Yolo desde github

# Modelos , ver pagina web: https://github.com/ultralytics/yolov5/releases

# yolov5n : modelo nano - 640 pixels
# yolov5s : modelo small - 640 pixels
# yolov5m : modelo medium - 640 pixels
# yolov5l : modelo large - 640 pixels
# yolov5x : modelo extra large - 640 pixels

# yolov5n6 : modelo nano - 1280 pixels
# yolov5s6 : modelo small - 1280 pixels
# yolov5m6 : modelo medium - 1280 pixels
# yolov5l6 : modelo large - 1280 pixels

# [INPUT] Indicar el modelo a leer
nombre_yolo = "yolov5s"

print('nombre modelo:',nombre_yolo)

try:
    model = torch.hub.load('ultralytics/yolov5', nombre_yolo)
    print('Se descargó modelo')
except:
    print('Revisar porque puede que no exista el archivo indicado')

In [3]:
# imprimir clases segun el modelo descargado - vienen a ser las clases
for (numero,clase) in model.names.items():
    print(numero,' : ',clase)

0  :  person
1  :  bicycle
2  :  car
3  :  motorcycle
4  :  airplane
5  :  bus
6  :  train
7  :  truck
8  :  boat
9  :  traffic light
10  :  fire hydrant
11  :  stop sign
12  :  parking meter
13  :  bench
14  :  bird
15  :  cat
16  :  dog
17  :  horse
18  :  sheep
19  :  cow
20  :  elephant
21  :  bear
22  :  zebra
23  :  giraffe
24  :  backpack
25  :  umbrella
26  :  handbag
27  :  tie
28  :  suitcase
29  :  frisbee
30  :  skis
31  :  snowboard
32  :  sports ball
33  :  kite
34  :  baseball bat
35  :  baseball glove
36  :  skateboard
37  :  surfboard
38  :  tennis racket
39  :  bottle
40  :  wine glass
41  :  cup
42  :  fork
43  :  knife
44  :  spoon
45  :  bowl
46  :  banana
47  :  apple
48  :  sandwich
49  :  orange
50  :  broccoli
51  :  carrot
52  :  hot dog
53  :  pizza
54  :  donut
55  :  cake
56  :  chair
57  :  couch
58  :  potted plant
59  :  bed
60  :  dining table
61  :  toilet
62  :  tv
63  :  laptop
64  :  mouse
65  :  remote
66  :  keyboard
67  :  cell phone
68  :  micro

In [4]:
# ver arquitectura de modelo 
model

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

# Prediccion de una sola imagen

Aqui se usará a modo de ejemplo una imagen disponible. Se puede cargar una imagen diferente en caso se desee.

In [15]:
# Indicar Imagen a usar

# [INPUT]
archivo = "bus.jpg"

# Leer imagen
img = cv2.imread(archivo, cv2.IMREAD_COLOR)
img = img[:,:,::-1] # Cambiar de BGR a RGB

print("archivo indicado:",archivo)
print('Dimension imagen:',img.shape)

archivo indicado: bus.jpg
Dimension imagen: (1080, 810, 3)


Se deja la imagen aqui en el bloque de comentarios:
<img src="bus.jpg" alt="Alternative text" />

In [6]:
# [Opcional] Aplicar escalamiento a imagen en caso sea pequeña

# Dimensiones de imagen
dims = img.shape
print('Dimension imagen inicio:',img.shape)

# [INPUT] Tamaño minimo de referencia
pixeles_min = 680

#######################

print('Cantidad de pixeles minimo por lado:',pixeles_min)

# Calcular escala
escala = max([pixeles_min/dims[0],pixeles_min/dims[1]])

# Solo en caso de requerirse escala hacia arriba
if(escala > 1.075):
    
    # Dimensiones
    width = int(dims[1]*escala)
    height = int(dims[0]*escala)
    dim = (width,height)
    
    # Hacer un resize de zoom (magnificacion)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
    
    print('\n*** Se aplicó escalamiento de imagen ***')
    print('Dimension imagen final:',img.shape)

    # guardar imagen en el archivo indicado
    archivo = archivo.split(".")[0] + "_res.jpg"
    cv2.imwrite(archivo, img[:,:,::-1]) # Opencv usa BGR
    print('Archivo con resize:',archivo)
    
else:
    print('No se requiere de escalar imagen')

Dimension imagen inicio: (1080, 810, 3)
Cantidad de pixeles minimo por lado: 680
No se requiere de escalar imagen


In [7]:
# Inferencia con modelo

resultado = model(archivo)

#resultado.xyxy[0]  # img1 predictions (tensor)
resultado.pandas().xyxy[0]  # img1 predictions (pandas)

,xmin,ymin,xmax,ymax,confidence,class,name
0,671.787781,395.371979,810.000000,878.361450,0.896172,0,person
1,220.657059,408.140900,346.167328,867.381287,0.870248,0,person
2,49.250805,389.990540,248.078201,912.458557,0.851563,0,person
3,12.650611,223.378433,809.707275,788.516357,0.849334,5,bus
4,0.045434,552.411316,67.882339,875.374634,0.534942,0,person


In [8]:
# Imagen salida - guardar a archivo

img_aux = img.copy()
datos = resultado.pandas().xyxy[0].copy()

# Filtrar segun confidencia de la prediccion
confidencia_minima = 0.175
datos = datos[datos.confidence > confidencia_minima].copy()
    
for ix in datos.index:
    datosz = datos.loc[ix,:].copy()
    min_point = (int(datosz.xmin),int(datosz.ymin))
    max_point = (int(datosz.xmax),int(datosz.ymax))
    img_aux = cv2.rectangle(img_aux,min_point,max_point,(255,255,255),1)
    
# ver imagen
plt.figure(figsize=(10,10))
plt.imshow(img_aux)
plt.show()  

# Nombre de archivo a guardar
filename_aux = archivo.split('.')[0] + "_pred" + ".jpg"

# Escribir
cv2.imwrite(filename_aux, img_aux[:,:,::-1]) # Opencv usa BGR
print('Se guardó en:',filename_aux)

Se guardó en: bus_pred.jpg


Se deja la imagen obtenida segun la prediccion del modelo:
<img src="bus_pred.jpg" alt="Alternative text" />

# Prediccion con webcam

Con el modelo leido se aplicará la prediccion en tiempo real. Al ejecutar el codigo, se usará la web cam que logre encontrar python segun el codigo. Se abrirá una ventana nueva segun Opencv y mediante el control **"s"** del teclado se podrá cerrar.

### Inferencia 1 - Deteccion de todos los objetos en camara web

In [9]:
# Prueba inferencia con imagen en tiempo real

import cv2

capture = cv2.VideoCapture(0)

while(capture.isOpened()):
    
    # leer frame del video / submuestreo
    subsampling_frames = 3      # cantidad submuestreo
    nn_frames = 0   # contador de submuestreo - Inicializar
    
    while(nn_frames < subsampling_frames):
        ret, frame = capture.read()
        nn_frames += 1  # contador submuestreo
    
    #################################
    
    # Inferencia con modelo
    resultado = model(frame)
    
    datos = resultado.pandas().xyxy[0].copy()
    
    titulo = 'Presione teclado "s" para finalizar grabacion' + ' - submuestreo: ' + str(subsampling_frames) 
    
    # Filtrar segun confidencia de la prediccion
    confidencia_minima = 0.175
    datos = datos[datos.confidence > confidencia_minima].copy()
    
    ###################
    
    # agregar rectangulo con objeto detectado
    for ix in datos.index:
        datosz = datos.loc[ix,:].copy()
        min_point = (int(datosz.xmin),int(datosz.ymin))
        max_point = (int(datosz.xmax),int(datosz.ymax))
        frame = cv2.rectangle(frame,min_point,max_point,(255,255,255),1)
    
    # mostrar imagen
    cv2.imshow(titulo,frame)
    
    #################################
    
    # break de bucle
    if (cv2.waitKey(1) == ord('s')):
        break


#############################

# liberar webcam
capture.release()

# cerrar ventana
cv2.destroyAllWindows()

### Inferencia 2 - Deteccion de solo personas en camara web

In [12]:
# Prueba inferencia con imagen en tiempo real - Filtrar por Persona

import cv2

capture = cv2.VideoCapture(0)

while(capture.isOpened()):
    
    # leer frame del video / submuestreo
    subsampling_frames = 3      # cantidad submuestreo
    nn_frames = 0   # contador de submuestreo - Inicializar
    
    while(nn_frames < subsampling_frames):
        ret, frame = capture.read()
        nn_frames += 1  # contador submuestreo
    
    #################################
    
    # Inferencia con modelo
    resultado = model(frame)
    
    datos = resultado.pandas().xyxy[0].copy()
    
    titulo = 'Presione teclado "s" para finalizar grabacion' + ' - submuestreo: ' + str(subsampling_frames) 
    
    # Filtrar segun confidencia de la prediccion
    confidencia_minima = 0.175
    datos = datos[datos.confidence > confidencia_minima].copy()
    
    #########################################

    # Filtrar
    datos = datos[datos.name.isin(["person"])]
    
    
    # agregar rectangulo con objeto detectado
    for ix in datos.index:
        datosz = datos.loc[ix,:].copy()
        min_point = (int(datosz.xmin),int(datosz.ymin))
        max_point = (int(datosz.xmax),int(datosz.ymax))
        frame = cv2.rectangle(frame,min_point,max_point,(255,255,255),1)
    
    # mostrar imagen
    cv2.imshow(titulo,frame)
    
    #################################
    
    # break de bucle
    if (cv2.waitKey(1) == ord('s')):
        break


#############################

# liberar webcam
capture.release()

# cerrar ventana
cv2.destroyAllWindows()